<img src="uva_seal.png">  


---
# DS5110 Team 3 Project Code
---
### Trey Briggs, Julie Crowe, Keenan Kunc
#### 25 February 2022
---

In [28]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col,isnan, when, count

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [117]:
# context manager
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("pspark_test") \
        .getOrCreate()

from pyspark.sql.functions import from_utc_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import date_format
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import min, max


In [3]:
path = 'tweets.csv'

df = spark.read.\
    format("csv").\
    option("delimiter", ";").\
    option("header", "true").\
    option("inferSchema", "true").load(path)
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- url: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- replies: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweets: string (nullable = true)
 |-- text: string (nullable = true)



In [4]:
df.show(10, False)

+-------------------+-------------+-------------------+----+----------------------+-------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                 |user         |fullname           |url |timestamp             |replies|likes|retweets|text                                                                                                                                                                                                                                  |
+-------------------+-------------+-------------------+----+----------------------+-------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Number of records  
 
Statistical summary of response variable  
Statistical summary of potential predictor variables (if there are a large number of predictors, select the top 10)  
Note: Summarize categorical variables with counts and percentages for each level and summarize numerical variables with mean/quantiles/standard deviation.  
Include up to five helpful graphs  

In [5]:
df.count()

40152515

Number of columns 

In [6]:
len(df.columns)

9

Statistical summary of potential predictor variables (if there are a large number of predictors, select the top 10)  
Note: Summarize categorical variables with counts and percentages for each level and summarize numerical variables with mean/quantiles/standard deviation.

In [7]:
#df.describe().show()

In [8]:
df.describe(['retweets']).show()

+-------+--------------------+
|summary|            retweets|
+-------+--------------------+
|  count|            16893092|
|   mean|  1.0487755215743861|
| stddev|   86.04669974795793|
|    min|                    |
|    max|ves  y sigue la r...|
+-------+--------------------+



In [9]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---+--------+--------+--------+---------+--------+--------+--------+--------+
| id|    user|fullname|     url|timestamp| replies|   likes|retweets|    text|
+---+--------+--------+--------+---------+--------+--------+--------+--------+
|241|22830824|23174141|36862014| 23256898|23258305|23258743|23259423|23260368|
+---+--------+--------+--------+---------+--------+--------+--------+--------+



In [10]:
cols = ['id', 'user', 'timestamp', 'replies', 'likes', 'retweets', 'text']

df2 = df.select(cols)


In [11]:
df_dropped = df2.na.drop()
df_dropped.cache()

DataFrame[id: string, user: string, timestamp: string, replies: string, likes: string, retweets: string, text: string]

In [12]:
df_dropped.show(10, False)

+-------------------+---------------+----------------------+-------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                 |user           |timestamp             |replies|likes|retweets|text                                                                                                                                                                                                                                  |
+-------------------+---------------+----------------------+-------+-----+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1132977055300300800|KamdemAbdiel   |2019-05-27 11:49:1

In [100]:
df_dropped.count()

16892858

In [18]:
path2 = 'bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv'

bit_price = spark.read.\
    format("csv").\
    option("delimiter", ",").\
    option("header", "true").\
    option("inferSchema", "true").load(path2)
bit_price.printSchema()

root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



In [19]:
bit_price.tail(5)
#bit_price.columns

['Timestamp',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume_(BTC)',
 'Volume_(Currency)',
 'Weighted_Price']

In [73]:
bit_price = bit_price.withColumn("StdDate", from_unixtime(col("Timestamp"), format='yyyy-MM-dd'))

In [74]:
bit_price.show(5)
#Beginning timestamp: 1325317920--Beginning of Bitcoin Prices data
#End timestamp: 1574467200--End of Tweets data

+----------+----+----+----+-----+------------+-----------------+--------------+----------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|   StdDate|
+----------+----+----+----+-----+------------+-----------------+--------------+----------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|2011-12-31|
|1325317980| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318040| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318100| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318160| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
+----------+----+----+----+-----+------------+-----------------+--------------+----------+
only showing top 5 rows



In [132]:
bit_price_filter = bit_price.filter((bit_price["Timestamp"] >= 1325317920) & (bit_price["Timestamp"] <= 1574467200))

In [124]:
bit_price_filter.show(5)

+----------+----+----+----+-----+------------+-----------------+--------------+----------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|   StdDate|
+----------+----+----+----+-----+------------+-----------------+--------------+----------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|2011-12-31|
|1325317980| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318040| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318100| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
|1325318160| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|2011-12-31|
+----------+----+----+----+-----+------------+-----------------+--------------+----------+
only showing top 5 rows



In [133]:
bit_price_na_drop = bit_price_filter.na.drop()
price_grp = bit_price_na_drop.groupBy("StdDate").avg("Weighted_Price").sort("StdDate", ascending = True)

In [126]:
price_grp.show(5)

+----------+-------------------+
|   StdDate|avg(Weighted_Price)|
+----------+-------------------+
|2011-12-31|     4.471602874575|
|2012-01-01|  4.806666666666667|
|2012-01-02|                5.0|
|2012-01-03|             5.2525|
|2012-01-04|  5.208159313655556|
+----------+-------------------+
only showing top 5 rows



In [127]:
df_date = df_dropped.withColumn("StdDate", date_format(col("timestamp"), "yyyy-MM-dd"))
df_date = df_date.withColumn('UnixTimestamp', unix_timestamp(col('StdDate'), 'yyyy-MM-dd'))

In [128]:
df_date.show(5)

+-------------------+-------------+--------------------+-------+-----+--------+--------------------+----------+-------------+
|                 id|         user|           timestamp|replies|likes|retweets|                text|   StdDate|UnixTimestamp|
+-------------------+-------------+--------------------+-------+-----+--------+--------------------+----------+-------------+
|1132977055300300800| KamdemAbdiel|2019-05-27 11:49:...|      0|    0|       0|È appena uscito u...|2019-05-27|   1558915200|
|1132977073402736640|    bitcointe|2019-05-27 11:49:...|      0|    0|       0|Cardano: Digitize...|2019-05-27|   1558915200|
|1132977023893139456|    3eyedbran|2019-05-27 11:49:...|      0|    2|       1|Another Test twee...|2019-05-27|   1558915200|
|1132977089089556481|DetroitCrypto|2019-05-27 11:49:...|      0|    0|       0|Current Crypto Pr...|2019-05-27|   1558915200|
|1132977092340191232| mmursaleen72|2019-05-27 11:49:...|      0|    0|       0|Spiv (Nosar Baz):...|2019-05-27|   1558

In [121]:
#df_date.agg(min(df_date["UnixTimestamp"])).show()
bit_price.agg(min(bit_price["Timestamp"])).show()

+--------------+
|min(Timestamp)|
+--------------+
|    1325317920|
+--------------+



In [122]:
#df_date.agg(max(df_date["UnixTimestamp"])).show()
bit_price.agg(max(bit_price["Timestamp"])).show()

+--------------+
|max(Timestamp)|
+--------------+
|    1617148800|
+--------------+



In [130]:
df_time = df_date.filter((df_date["UnixTimestamp"] >= 1325317920) & (df_date["UnixTimestamp"] <= 1574467200))

In [131]:
df_time.count()

16889437

In [134]:
final_df = df_time.join(price_grp,df_time["StdDate"] == price_grp["StdDate"],"left")

In [135]:
final_df.show(5)

+------------------+---------------+--------------------+-------+-----+--------+--------------------+----------+-------------+----------+-------------------+
|                id|           user|           timestamp|replies|likes|retweets|                text|   StdDate|UnixTimestamp|   StdDate|avg(Weighted_Price)|
+------------------+---------------+--------------------+-------+-----+--------+--------------------+----------+-------------+----------+-------------------+
|312067116096438273|JamesVancePhoto|2013-03-14 04:06:...|      0|    0|       1|@Btc_ImFree mixta...|2013-03-14|   1363219200|2013-03-14|  47.36082563895725|
|312316656590389248|     falloutkez|2013-03-14 20:37:...|      0|    0|       0|Ellen btc is at 5:00|2013-03-14|   1363219200|2013-03-14|  47.36082563895725|
|312365399960346624|    mtgoxtrades|2013-03-14 23:51:...|      0|    0|       0|Fri Mar 15 00:50:...|2013-03-14|   1363219200|2013-03-14|  47.36082563895725|
|312294576759123968|      bcchanger|2013-03-14 19:10

In [136]:
cols_keep = ["id", "user", "timestamp", "replies", "likes", 'retweets', 'text', 'avg(Weighted_Price)']
final = final_df.select(cols_keep)

In [137]:
final.show(5)

+------------------+---------------+--------------------+-------+-----+--------+--------------------+-------------------+
|                id|           user|           timestamp|replies|likes|retweets|                text|avg(Weighted_Price)|
+------------------+---------------+--------------------+-------+-----+--------+--------------------+-------------------+
|312067116096438273|JamesVancePhoto|2013-03-14 04:06:...|      0|    0|       1|@Btc_ImFree mixta...|  47.36082563895725|
|312316656590389248|     falloutkez|2013-03-14 20:37:...|      0|    0|       0|Ellen btc is at 5:00|  47.36082563895725|
|312365399960346624|    mtgoxtrades|2013-03-14 23:51:...|      0|    0|       0|Fri Mar 15 00:50:...|  47.36082563895725|
|312294576759123968|      bcchanger|2013-03-14 19:10:...|      0|    0|       0|lamaestro wants t...|  47.36082563895725|
|312172551017205760|      bcchanger|2013-03-14 11:05:...|      0|    0|       0|Cash4Coin wants t...|  47.36082563895725|
+------------------+----

In [138]:
#final.write.csv("output.csv")